In [21]:
import tensorflow as tf 
from tensorflow.keras.datasets import mnist
tf.compat.v1.disable_eager_execution()
(x_train,y_train), (x_test,y_test) = mnist.load_data()
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)
x_train.shape , y_train.shape

((60000, 28, 28), (60000, 10))

In [22]:
# all the variables 
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_units_conv1 = 32
n_units_conv2 = 64
conv1_filter_k = 5
conv2_filter_k = 5
stride_conv1 = 1
stride_conv2 = 1
max_pool1 = 2
max_pool2 = 2

n_units_hidden = 1024
n_out = 10
n_input_for_1st_hidden_layer = (input_width//(max_pool1*max_pool2))*(input_height//(max_pool1*max_pool2))*n_units_conv2

In [23]:
# initializing weghts and biases
weights = {'wc1': tf.Variable(tf.random.normal([conv1_filter_k,conv1_filter_k,input_channels,n_units_conv1])),
           'wc2': tf.Variable(tf.random.normal([conv2_filter_k,conv2_filter_k,n_units_conv1,n_units_conv2])),
           'wh1': tf.Variable(tf.random.normal([n_input_for_1st_hidden_layer,n_units_hidden])),
           'wo': tf.Variable(tf.random.normal([n_units_hidden,n_out]))
           }
biases = {'bc1': tf.Variable(tf.random.normal([n_units_conv1])),
          'bc2': tf.Variable(tf.random.normal([n_units_conv2])),
          'bh1': tf.Variable(tf.random.normal([n_units_hidden])),
          'bo': tf.Variable(tf.random.normal([n_out]))
          }

In [24]:
#convolutional layer function
def conv(x,weights,bias,strides = 1):
    out = tf.nn.conv2d(x,weights,padding='SAME',strides=[1,strides,strides,1])
    out = tf.nn.bias_add(out,bias)
    return out
# pooling  layer function 
def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding='SAME',ksize=[1,k,k,1],strides=[1,k,k,1])

In [25]:
# defining the forward propagation function 
def cnn(x,weights,biases):
    x = tf.reshape(x,shape=[-1,input_height,input_width,input_channels])
    conv1 = conv(x,weights['wc1'],biases['bc1'])
    conv1_pool = maxpooling(conv1,max_pool1)

    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'])
    conv2_pool = maxpooling(conv2,max_pool2)

    hidden_input = tf.reshape(conv2_pool,shape=[-1,n_input_for_1st_hidden_layer])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    hidden_output = tf.nn.relu(hidden_output_before_activation)
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output

In [17]:
x = tf.compat.v1.placeholder("float",[None,input_width,input_height])
y = tf.compat.v1.placeholder(tf.int32,[None,n_out])
pred = cnn(x,weights,biases)

In [18]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01)
variables = [weights['wc1'],weights['wc2'],weights['wh1'],weights['wo'],biases['bc1'],biases['bc2'],biases['bh1'],biases['bo']]
optimize = optimizer.minimize(cost,var_list=variables)

In [27]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())
batch_size = 32  # or a smaller value
for i in range(15):
    for start in range(0, len(x_train), batch_size):
        end = start + batch_size
        batch_x = x_train[start:end]
        batch_y = y_train[start:end]
        c, _ = sess.run([cost, optimize], feed_dict={x: batch_x, y: batch_y})
        print(i,"no iteration and cost is - ",c)

0 no iteration and cost is -  24777346.0
0 no iteration and cost is -  20970696.0
0 no iteration and cost is -  12017158.0
0 no iteration and cost is -  17016162.0
0 no iteration and cost is -  28129072.0
0 no iteration and cost is -  24486900.0
0 no iteration and cost is -  10180292.0
0 no iteration and cost is -  21607284.0
0 no iteration and cost is -  36089480.0
0 no iteration and cost is -  13142097.0
0 no iteration and cost is -  13275113.0
0 no iteration and cost is -  4336413.5
0 no iteration and cost is -  9269582.0
0 no iteration and cost is -  5177683.0
0 no iteration and cost is -  4551863.0
0 no iteration and cost is -  12193739.0
0 no iteration and cost is -  7972169.0
0 no iteration and cost is -  3721560.8
0 no iteration and cost is -  987443.1
0 no iteration and cost is -  2327528.5
0 no iteration and cost is -  2368352.2
0 no iteration and cost is -  2585705.0
0 no iteration and cost is -  5469733.5
0 no iteration and cost is -  4219305.5
0 no iteration and cost is - 

In [28]:
predictions = tf.argmax(pred,1)
corr_labels = tf.argmax(y,1)
corr_preds = tf.equal(predictions,corr_labels)
prediction_eval, labels, correct_pred = sess.run([predictions,corr_labels,corr_preds],feed_dict = {x : x_test,y : y_test})
accuracy = correct_pred.sum()/x_test.shape[0]
accuracy*100

98.39